In [1]:
# Импортируем необходимые библиотеки
import pandas as pd
from datetime import datetime

In [2]:
# Считаем файл и отобразим первые 5 строк
df = pd.read_csv('data.csv', encoding='windows-1251', sep=';')
df.head()

,Номер,Дата создания,Дата оплаты,Title,Статус,Заработано,Город,Платежная система
0,1062823,01.12.2019 10:50,01.12.2019 10:52,Курс обучения «Эксперт»,Завершен,29597.5,Чита,Сбербанк эквайринг
1,1062855,01.12.2019 20:53,01.12.2019 21:27,Курс обучения «Эксперт»,Завершен,17450.3,Краснодар,Яндекс.Касса
2,1062856,01.12.2019 21:43,NaN,Курс обучения «Специалист»,Отменен,0.0,NaN,NaN
3,1062880,03.12.2019 0:18,NaN,Курс обучения «Консультант»,Отменен,0.0,г.Москва и Московская область,NaN
4,1062899,03.12.2019 21:43,NaN,Курс обучения «Эксперт»,Отменен,0.0,г.Москва и Московская область,NaN


In [3]:
# Посмотрим на размерность данных
df.shape

(292, 8)

In [4]:
# Посмотрим какие типы данных имеются в датасете
df.dtypes

Номер                  int64
Дата создания         object
Дата оплаты           object
Title                 object
Статус                object
Заработано           float64
Город                 object
Платежная система     object
dtype: object

In [5]:
# Посмотрим на итоговую статистику по количественным переменным
df.describe()

,Номер,Заработано
count,2.920000e+02,292.000000
mean,1.063745e+06,3397.615034
std,4.438688e+02,5771.572829
min,1.062823e+06,0.000000
25%,1.063608e+06,0.000000
50%,1.063698e+06,2935.440000
75%,1.063807e+06,2935.440000
max,1.064796e+06,42750.000000


In [6]:
# Выведем названия колонок в датасете
df.columns

Index(['Номер', 'Дата создания', 'Дата оплаты', 'Title', 'Статус',
       'Заработано', 'Город', 'Платежная система'],
      dtype='object')

In [7]:
# Переименуем названия колонок на английский язык в единообразном стиле
df = df.rename(columns={
    'Номер': 'number',
    'Дата создания': 'create_date',
    'Дата оплаты': 'payment_date',
    'Title': 'title',
    'Статус': 'status',
    'Заработано': 'money', 
    'Город': 'city',
    'Платежная система': 'payment_system'
})

In [8]:
# Отобразим данные и убедимся что все столбцы переименованы
df.tail()

,number,create_date,payment_date,title,status,money,city,payment_system
287,1064720,30.12.2019 9:42,30.12.2019 12:49,Подписка «ОНЛАЙН ДИЕТОЛОГ» с ежемесячным автос...,Завершен,2935.44,Самарская область,Яндекс.Касса
288,1064724,30.12.2019 11:32,NaN,Курс обучения «Консультант»,Отменен,0.00,NaN,NaN
289,1064775,31.12.2019 2:17,31.12.2019 2:22,Курс обучения «Консультант»,Завершен,7423.92,NaN,"Сбербанк эквайринг,Бонусный счет"
290,1064793,31.12.2019 16:40,01.01.2020 14:29,Подписка «ОНЛАЙН ДИЕТОЛОГ» с ежемесячным автос...,Завершен,2935.44,Республика Карелия,Яндекс.Касса
291,1064796,31.12.2019 17:29,31.12.2019 17:32,Курс от Школы Диетологов. Повышение квалификац...,Завершен,9898.56,Уфа,"Сбербанк эквайринг,Бонусный счет"


#### Создадим переменную all_money перед тем как начать дальнейшее преобразование данных и вычисления. Это необходимо для того, чтобы не ошибится с расчетами и не потерять данные в процессе аналитики. С этой переменной можно будет сравнить итоговый результат.

In [9]:
all_money = df.money.sum()
all_money

992103.5900000001

Решим следующую задачу. Сколько заработано денег на курсах с группировкой по названию курса и разбивкой по городам?

In [10]:
# Создадим переменную money_by_city в которую сохраним сгруппированные и агрегированные данные
money_by_city = df \
    .groupby(['title', 'city'], as_index=False) \
    .aggregate({'money': 'sum'}) \
    .sort_values('money', ascending=False)

In [11]:
money_by_city.head()

,title,city,money
51,Курс обучения «Эксперт»,г.Санкт-Петербург и Ленинградская область,59195.00
156,Подписка «ОНЛАЙН ДИЕТОЛОГ» с ежемесячным автос...,г.Москва и Московская область,46967.04
39,Курс обучения «Эксперт»,Балхаш,42750.00
95,Подписка «ОНЛАЙН ДИЕТОЛОГ» с ежемесячным автос...,Краснодарский край,38169.78
30,Курс обучения «Специалист»,Краснодар,29695.70


In [12]:
# Сохраняем датафрейм в csv файл
money_by_city.to_csv('money_by_city.csv', index=False)

Решим следующую задачу. Необходимо посчитать по каждому продукту сколько суммарных денег и какое количество завершенных заказов, т.е. какое количество заказов позволило заработать определенное количество денег.

In [13]:
# Создадим переменную money_title в которую сохраним сгрупированные и агрегированные данные
money_title = df \
    .query("status == 'Завершен'") \
    .groupby('title', as_index=False) \
    .aggregate({'money': 'sum', 'number': 'count'}) \
    .sort_values('money', ascending=False) \
    .rename(columns={'number': 'success_orders'})

money_title.head()

,title,money,success_orders
5,Подписка «ОНЛАЙН ДИЕТОЛОГ» с ежемесячным автос...,366947.20,125
0,Курс обучения «Консультант»,208163.49,31
1,Курс обучения «Специалист»,160862.64,7
2,Курс обучения «Эксперт»,148992.80,5
4,Курс от Школы Диетологов. Повышение квалификац...,88384.92,9


In [14]:
# Создадим переменную today_day с текущей датой и создадим заготовку для названия файла при записи
today_day = datetime.today().strftime('%Y-%m-%d')
file_name = 'money_title_{}.csv'
file_name = file_name.format(today_day)

Проверим сохранилась ли логическая структура данных, а именно сравним сумму всех денег в датафрейме и сумму всех денег в преобразованных данных. Если условие выполняется, то записываем файл в csv. Если условие не соблюдается - выводим сообщение об ошибке.

In [15]:
# Проверка условия и запись файла
if int(money_title.money.sum()) == int(all_money):
    print('OK! File {} is written.'.format(file_name))
    money_title.to_csv(file_name, index=False)
else:
    print('ERROR!')

OK! File money_title_2021-11-17.csv is written.
